In [1]:
# === GOLD schema → DataFrame variables (cvss_v3, dim_cve, mv_cve_all_cvss, ...) ===
import os, re
import psycopg2
import pandas as pd
from types import SimpleNamespace

DB_CONFIG = {
    "user": os.getenv("PG_USER", "postgres"),
    "password": os.getenv("PG_PASSWORD", "tip_pwd"),
    "host": os.getenv("PG_HOST", "localhost"),
    "port": os.getenv("PG_PORT", "5432"),
    "database": os.getenv("PG_DB", "tip"),
}
SCHEMA = "gold"

def connect_db():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✓ DB connected")
        return conn
    except Exception as e:
        raise RuntimeError(f"DB connection failed: {e}")

def list_gold_relations(conn):
    q = """
    WITH base_views AS (
        SELECT table_name AS name, table_type AS kind
        FROM information_schema.tables
        WHERE table_schema = %s AND table_type IN ('BASE TABLE','VIEW')
    ),
    matviews AS (
        SELECT matviewname AS name, 'MATERIALIZED VIEW'::text AS kind
        FROM pg_matviews
        WHERE schemaname = %s
    )
    SELECT name, kind
    FROM base_views
    UNION ALL
    SELECT name, kind
    FROM matviews
    ORDER BY name;
    """
    return pd.read_sql_query(q, conn, params=[SCHEMA, SCHEMA])

def _to_valid_identifier(name: str) -> str:
    """Turn relation name into a safe Python identifier (for attribute/variable)."""
    ident = re.sub(r'\W+', '_', name)  # non-word → _
    ident = re.sub(r'^(\d)', r'_\1', ident)  # no leading digit
    return ident.lower()

def load_gold_as_namespace(conn, limit=None, verbose=True):
    """
    Loads every relation from `gold` into a SimpleNamespace with attributes named
    after the relation (safe identifiers). Also returns a {attr_name: relation_name} map.
    """
    rels = list_gold_relations(conn)
    if rels.empty:
        print("⚠️ No relations found in schema 'gold'")
        return SimpleNamespace(), {}

    ns = SimpleNamespace()
    name_map = {}  # attr_name -> real relation name
    used = set()

    for _, row in rels.iterrows():
        rel_name = row["name"]
        kind = row["kind"]
        fq = f'{SCHEMA}."{rel_name}"'
        sql = f"SELECT * FROM {fq}" + (f" LIMIT {int(limit)}" if limit else "")
        try:
            df = pd.read_sql_query(sql, conn)
        except Exception as e:
            print(f"✗ Load error {rel_name} ({kind}): {e}")
            continue

        # choose a variable/attribute name
        base = _to_valid_identifier(rel_name)
        cand = base
        i = 1
        while cand in used:
            i += 1
            cand = f"{base}_{i}"
        setattr(ns, cand, df)
        name_map[cand] = rel_name
        used.add(cand)

        if verbose:
            print(f"✓ {cand:<22} ← {rel_name:<22} [{kind:<18}] rows={len(df)} cols={len(df.columns)}")

    return ns, name_map

def inject_into_globals(ns, g=None):
    """Convenience: inject namespace DataFrames as top-level variables (Jupyter)."""
    g = g if g is not None else globals()
    for k, v in ns.__dict__.items():
        g[k] = v
    print(f"✓ Injected {len(ns.__dict__)} datasets into globals()")

# --- Run this cell to load & expose variables like cvss_v3, dim_cve, mv_cve_all_cvss ---
conn = connect_db()
ns, name_map = load_gold_as_namespace(conn, limit=None, verbose=True)
inject_into_globals(ns)   # now you can call cvss_v3.head(10), dim_cve.head(10), etc.
conn.close()

# Optional: show mapping if any names were adjusted
pd.DataFrame([{"python_var": k, "relation": v} for k, v in name_map.items()])

✓ DB connected


C:\Users\hamza\AppData\Local\Temp\ipykernel_48616\3143261205.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(q, conn, params=[SCHEMA, SCHEMA])
C:\Users\hamza\AppData\Local\Temp\ipykernel_48616\3143261205.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


✓ agg_cia_impact_analysis ← agg_cia_impact_analysis [VIEW              ] rows=32 cols=9
✓ agg_cve_age_analysis   ← agg_cve_age_analysis   [VIEW              ] rows=77577 cols=11
✓ agg_cve_by_category    ← agg_cve_by_category    [VIEW              ] rows=16 cols=13
✓ agg_cve_by_year        ← agg_cve_by_year        [VIEW              ] rows=30 cols=16
✓ agg_cvss_version_comparison ← agg_cvss_version_comparison [VIEW              ] rows=6902 cols=16
✓ agg_exploitability_analysis ← agg_exploitability_analysis [VIEW              ] rows=12 cols=12
✓ agg_monthly_cve_trends ← agg_monthly_cve_trends [VIEW              ] rows=179 cols=12
✓ agg_product_vulnerability ← agg_product_vulnerability [VIEW              ] rows=56456 cols=17
✓ agg_top_attack_vectors ← agg_top_attack_vectors [VIEW              ] rows=4 cols=10
✓ agg_vendor_product_matrix ← agg_vendor_product_matrix [VIEW              ] rows=56456 cols=15
✓ agg_vendor_risk_score  ← agg_vendor_risk_score  [VIEW              ] rows=12077 cols

,python_var,relation
0,agg_cia_impact_analysis,agg_cia_impact_analysis
1,agg_cve_age_analysis,agg_cve_age_analysis
2,agg_cve_by_category,agg_cve_by_category
3,agg_cve_by_year,agg_cve_by_year
4,agg_cvss_version_comparison,agg_cvss_version_comparison
5,agg_exploitability_analysis,agg_exploitability_analysis
6,agg_monthly_cve_trends,agg_monthly_cve_trends
7,agg_product_vulnerability,agg_product_vulnerability
8,agg_top_attack_vectors,agg_top_attack_vectors
9,agg_vendor_product_matrix,agg_vendor_product_matrix


In [3]:
bridge_cve_products.head(10)   

,bridge_id,cve_id,product_id,created_at
0,1,CVE-2010-0315,1,2025-10-17 22:24:13.455246
1,2,CVE-2010-0249,2,2025-10-17 22:24:13.455246
2,3,CVE-2010-0249,3,2025-10-17 22:24:13.455246
3,4,CVE-2010-0249,4,2025-10-17 22:24:13.455246
4,5,CVE-2010-0249,5,2025-10-17 22:24:13.455246
5,6,CVE-2010-0249,6,2025-10-17 22:24:13.455246
6,7,CVE-2010-0249,7,2025-10-17 22:24:13.455246
7,8,CVE-2010-0249,8,2025-10-17 22:24:13.455246
8,9,CVE-2010-0280,9,2025-10-17 22:24:13.455246
9,10,CVE-2010-0280,10,2025-10-17 22:24:13.455246


In [65]:
cvss_v2.head(10)

,cvss_v2_id,cve_id,source_id,cvss_score,cvss_severity,cvss_vector,cvss_v2_av,cvss_v2_ac,cvss_v2_au,cvss_v2_c,cvss_v2_i,cvss_v2_a,cvss_exploitability_score,cvss_impact_score,created_at
0,1,CVE-2010-0315,144,5.0,MEDIUM,AV:N/AC:L/Au:N/C:P/I:N/A:N,N,L,N,P,N,N,10.0,2.9,2025-10-17 18:48:59.902221
1,2,CVE-2010-0249,144,9.3,HIGH,AV:N/AC:M/Au:N/C:C/I:C/A:C,N,M,N,C,C,C,8.6,10.0,2025-10-17 18:48:59.902221
2,3,CVE-2010-0280,144,9.3,HIGH,AV:N/AC:M/Au:N/C:C/I:C/A:C,N,M,N,C,C,C,8.6,10.0,2025-10-17 18:48:59.902221
3,4,CVE-2010-0316,144,9.3,HIGH,AV:N/AC:M/Au:N/C:C/I:C/A:C,N,M,N,C,C,C,8.6,10.0,2025-10-17 18:48:59.902221
4,5,CVE-2010-0317,144,7.8,HIGH,AV:N/AC:L/Au:N/C:N/I:N/A:C,N,L,N,N,N,C,10.0,6.9,2025-10-17 18:48:59.902221
5,6,CVE-2010-0318,144,6.9,MEDIUM,AV:L/AC:M/Au:N/C:C/I:C/A:C,L,M,N,C,C,C,3.4,10.0,2025-10-17 18:48:59.902221
6,7,CVE-2010-0319,144,4.3,MEDIUM,AV:N/AC:M/Au:N/C:N/I:P/A:N,N,M,N,N,P,N,8.6,2.9,2025-10-17 18:48:59.902221
7,8,CVE-2010-0320,144,4.3,MEDIUM,AV:N/AC:M/Au:N/C:N/I:P/A:N,N,M,N,N,P,N,8.6,2.9,2025-10-17 18:48:59.902221
8,9,CVE-2010-0321,144,4.3,MEDIUM,AV:N/AC:M/Au:N/C:N/I:P/A:N,N,M,N,N,P,N,8.6,2.9,2025-10-17 18:48:59.902221
9,10,CVE-2010-0322,144,7.5,HIGH,AV:N/AC:L/Au:N/C:P/I:P/A:P,N,L,N,P,P,P,10.0,6.4,2025-10-17 18:48:59.902221


In [66]:
cvss_v3.head(10)

,cvss_v3_id,cve_id,source_id,cvss_version,cvss_score,cvss_severity,cvss_vector,cvss_v3_base_av,cvss_v3_base_ac,cvss_v3_base_pr,cvss_v3_base_ui,cvss_v3_base_s,cvss_v3_base_c,cvss_v3_base_i,cvss_v3_base_a,cvss_exploitability_score,cvss_impact_score,created_at
0,1,CVE-2010-0249,144,CVSS 3.1,8.8,HIGH,CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H,N,L,N,R,U,H,H,H,2.8,5.9,2025-10-17 18:49:09.582708
1,2,CVE-2010-0036,11,CVSS 3.1,7.8,HIGH,CVSS:3.1/AV:L/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H,L,L,N,R,U,H,H,H,1.8,5.9,2025-10-17 18:49:09.582708
2,3,CVE-2010-0037,11,CVSS 3.1,8.8,HIGH,CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H,N,L,N,R,U,H,H,H,2.8,5.9,2025-10-17 18:49:09.582708
3,4,CVE-2010-0232,144,CVSS 3.1,7.8,HIGH,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,L,L,L,N,U,H,H,H,1.8,5.9,2025-10-17 18:49:09.582708
4,5,CVE-2010-0232,11,CVSS 3.1,7.8,HIGH,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,L,L,L,N,U,H,H,H,1.8,5.9,2025-10-17 18:49:09.582708
5,6,CVE-2025-21707,144,CVSS 3.1,5.5,MEDIUM,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:N/I:N/A:H,L,L,L,N,U,N,N,H,1.8,3.6,2025-10-17 18:49:09.582708
6,7,CVE-2010-0378,144,CVSS 3.1,8.8,HIGH,CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H,N,L,N,R,U,H,H,H,2.8,5.9,2025-10-17 18:49:09.582708
7,8,CVE-2010-0248,11,CVSS 3.1,8.1,HIGH,CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:H/A:H,N,H,N,N,U,H,H,H,2.2,5.9,2025-10-17 18:49:09.582708
8,9,CVE-2010-0467,144,CVSS 3.1,5.8,MEDIUM,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:C/C:L/I:N/A:N,N,L,N,N,C,L,N,N,3.9,1.4,2025-10-17 18:49:09.582708
9,10,CVE-2025-9559,83,CVSS 3.1,6.5,MEDIUM,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:N/A:N,N,L,L,N,U,H,N,N,NaN,NaN,2025-10-17 18:49:09.582708


In [67]:
cvss_v4.head(10)

,cvss_v4_id,cve_id,source_id,cvss_score,cvss_severity,cvss_vector,cvss_v4_av,cvss_v4_at,cvss_v4_ac,cvss_v4_vc,cvss_v4_vi,cvss_v4_va,cvss_v4_sc,cvss_v4_si,cvss_v4_sa,created_at
0,1,CVE-2025-62496,14,7.1,HIGH,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:H/VA...,A,P,H,H,H,L,H,H,L,2025-10-17 18:49:29.980124
1,2,CVE-2025-62496,96,7.1,HIGH,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:H/VA...,A,P,H,H,H,L,H,H,L,2025-10-17 18:49:29.980124
2,3,CVE-2025-62495,14,7.1,HIGH,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:H/VA...,A,P,H,H,H,L,H,H,L,2025-10-17 18:49:29.980124
3,4,CVE-2025-62495,96,7.1,HIGH,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:H/VA...,A,P,H,H,H,L,H,H,L,2025-10-17 18:49:29.980124
4,5,CVE-2025-62494,14,7.1,HIGH,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:H/VA...,A,P,H,H,H,L,H,H,L,2025-10-17 18:49:29.980124
5,6,CVE-2025-62494,96,7.1,HIGH,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:H/VA...,A,P,H,H,H,L,H,H,L,2025-10-17 18:49:29.980124
6,7,CVE-2025-62493,14,5.9,MEDIUM,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:L/VA...,A,P,H,H,L,L,H,L,L,2025-10-17 18:49:29.980124
7,8,CVE-2025-62493,96,5.9,MEDIUM,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:L/VA...,A,P,H,H,L,L,H,L,L,2025-10-17 18:49:29.980124
8,9,CVE-2025-62492,14,5.9,MEDIUM,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:L/VA...,A,P,H,H,L,L,H,L,L,2025-10-17 18:49:29.980124
9,10,CVE-2025-62492,96,5.9,MEDIUM,CVSS:4.0/AV:A/AC:H/AT:P/PR:L/UI:P/VC:H/VI:L/VA...,A,P,H,H,L,L,H,L,L,2025-10-17 18:49:29.980124


In [68]:
dim_cve.head(10)

,cve_id,title,description,category,predicted_category,published_date,last_modified,loaded_at,cve_year,remotely_exploit,source_identifier,created_at
0,CVE-1999-0019,OpenSSL statd Remote Code Execution (RCE),The following products are affected byCVE-1999...,undefined,None,1996-04-24 04:00:00,2025-04-03 01:03:00,2025-10-17 16:55:29.830017,1996,None,cve@mitre.org,2025-10-17 18:48:33.935726
1,CVE-1999-0022,Oracle rdist Root Buffer Overflow Vulnerability,The following products are affected byCVE-1999...,undefined,None,1996-07-03 04:00:00,2025-04-03 01:03:00,2025-10-17 16:55:29.830017,1996,False,cve@mitre.org,2025-10-17 18:48:33.935726
2,CVE-1999-0023,Apache rdist Buffer Overflow Root Privilege Es...,The following products are affected byCVE-1999...,undefined,None,1996-07-24 04:00:00,2025-04-03 01:03:00,2025-10-17 16:55:29.830017,1996,False,cve@mitre.org,2025-10-17 18:48:33.935726
3,CVE-1999-0032,"""lpr Buffer Overflow Root Execution""",The following products are affected byCVE-1999...,undefined,None,1996-10-25 04:00:00,2025-04-03 01:03:00,2025-10-17 16:55:29.830017,1996,False,cve@mitre.org,2025-10-17 18:48:33.935726
4,CVE-1999-0043,Inn Daemon Command Injection Vulnerability,The following products are affected byCVE-1999...,undefined,None,1996-12-04 05:00:00,2025-04-03 01:03:00,2025-10-17 16:55:29.830017,1996,None,cve@mitre.org,2025-10-17 18:48:33.935726
5,CVE-1999-0044,IRIX fsdump Local Privilege Escalation Vulnera...,The following products are affected byCVE-1999...,undefined,None,1996-12-03 05:00:00,2025-04-03 01:03:00,2025-10-17 16:55:29.830017,1996,False,cve@mitre.org,2025-10-17 18:48:33.935726
6,CVE-1999-0045,Apache Web Host Command Injection,The following products are affected byCVE-1999...,undefined,None,1996-12-10 05:00:00,2025-04-03 01:03:00,2025-10-17 16:55:29.830017,1996,None,cve@mitre.org,2025-10-17 18:48:33.935726
7,CVE-1999-0050,HP-UX newgrp Buffer Overflow Vulnerability,The following products are affected byCVE-1999...,undefined,None,1996-12-01 05:00:00,2025-04-03 01:03:00,2025-10-17 16:55:29.830017,1996,False,cve@mitre.org,2025-10-17 18:48:33.935726
8,CVE-1999-0066,Apache AnyForm Remote Code Execution,The following products are affected byCVE-1999...,undefined,None,1995-07-31 04:00:00,2025-04-03 01:03:00,2025-10-17 16:51:01.228214,1995,None,cve@mitre.org,2025-10-17 18:48:33.935726
9,CVE-1999-0067,Apache phf Command Execution Vulnerability,The following products are affected byCVE-1999...,undefined,None,1996-03-20 05:00:00,2025-04-03 01:03:00,2025-10-17 16:55:29.830017,1996,None,cve@mitre.org,2025-10-17 18:48:33.935726


In [69]:
dim_cvss_source.head(10)

,source_id,source_name,created_at
0,1,001d69cf-3fc9-4203-93fb-9865b54e05b2,2025-10-17 18:48:33.519713
1,2,02762ae7-200e-4b20-9b2b-a77d5b8fc4cb,2025-10-17 18:48:33.519713
2,3,05b98450-cf83-4905-9546-e47a66a8fec2,2025-10-17 18:48:33.519713
3,4,09832df1-09c1-45b4-8a85-16c601d30feb,2025-10-17 18:48:33.519713
4,5,0beee27a-7d8c-424f-8e46-ac453fa147e6,2025-10-17 18:48:33.519713
5,6,0bf9931a-6ebf-4f48-bd14-39ee5e1d61f8,2025-10-17 18:48:33.519713
6,7,0fc0942c-577d-436f-ae8e-945763c79b02,2025-10-17 18:48:33.519713
7,8,103e4ec9-0a87-450b-af77-479448ddef11,2025-10-17 18:48:33.519713
8,9,10b61619-3869-496c-8a1e-f291b0e71e3f,2025-10-17 18:48:33.519713
9,10,13061848-ea10-403d-bd75-c83a022c2891,2025-10-17 18:48:33.519713


In [70]:
dim_products.head(10)

,product_id,vendor_id,product_name,total_cves,first_cve_date,last_cve_date,created_at
0,1,1,chrome,1247,2010-01-14 19:30:00,2025-07-30 02:17:00,2025-10-17 18:48:53.765877
1,2,2,windows_7,981,2010-01-12 17:30:00,2023-05-31 19:15:00,2025-10-17 18:48:53.765877
2,3,2,windows_server_2008,1161,2010-01-13 19:30:00,2025-07-08 17:16:00,2025-10-17 18:48:53.765877
3,4,2,internet_explorer,533,2010-01-15 17:30:00,2018-09-13 00:29:00,2025-10-17 18:48:53.765877
4,5,2,windows_2000,61,1997-01-01 05:00:00,2014-04-23 20:55:00,2025-10-17 18:48:53.765877
5,6,2,windows_server_2003,444,2010-01-15 17:30:00,2020-02-20 15:15:00,2025-10-17 18:48:53.765877
6,7,2,windows_vista,731,2010-01-12 17:30:00,2020-02-20 15:15:00,2025-10-17 18:48:53.765877
7,8,2,windows_xp,752,2010-01-13 19:30:00,2020-02-20 15:15:00,2025-10-17 18:48:53.765877
8,9,3,lib3ds,1,2010-01-15 17:30:00,2010-01-15 17:30:00,2025-10-17 18:48:53.765877
9,10,1,google_sketchup,2,2010-01-15 17:30:00,2010-01-15 17:30:00,2025-10-17 18:48:53.765877


In [71]:
dim_vendor.head(10)

,vendor_id,vendor_name,total_products,total_cves,first_cve_date,last_cve_date,created_at
0,1,Google,57,2971,2010-01-14 19:30:00,2025-08-04 02:15:00,2025-10-17 18:48:52.493862
1,2,Microsoft,480,22865,1995-01-01 05:00:00,2025-07-30 02:17:00,2025-10-17 18:48:52.493862
2,3,Jan_eric_krprianidis,1,1,2010-01-15 17:30:00,2010-01-15 17:30:00,2025-10-17 18:48:52.493862
3,4,Novell,53,242,1993-09-16 04:00:00,2025-06-18 16:15:00,2025-10-17 18:48:52.493862
4,5,Freebsd,3,112,1990-05-09 04:00:00,2025-01-30 05:15:00,2025-10-17 18:48:52.493862
5,6,Docmint,1,1,2010-01-15 19:30:00,2010-01-15 19:30:00,2025-10-17 18:48:52.493862
6,7,X10media,1,1,2010-01-15 19:30:00,2010-01-15 19:30:00,2025-10-17 18:48:52.493862
7,8,Jamit,1,1,2010-01-15 19:30:00,2010-01-15 19:30:00,2025-10-17 18:48:52.493862
8,9,Typo3,22,249,2010-01-15 19:30:00,2025-07-22 11:15:00,2025-10-17 18:48:52.493862
9,10,Matthias_karr,1,1,2010-01-15 19:30:00,2010-01-15 19:30:00,2025-10-17 18:48:52.493862
